In [ ]:
!pip install gradio
!pip install google-generativeai

In [ ]:
import gradio as gr
import json
import os
import re
from pathlib import Path
from typing import Dict, List, Optional
import google.generativeai as genai
from enum import Enum, auto

# Initialize Gemini 2.0 Flash
genai.configure(api_key="your gemini api key")  # Note: This should be your actual API key
model = genai.GenerativeModel('gemini-2.0-flash')

class Language(Enum):
    PYTHON = auto()
    JAVASCRIPT = auto()
    JAVA = auto()
    CSHARP = auto()
    GO = auto()

class CodeAnalysisTools:
    @staticmethod
    def detect_language(file_path: str) -> Optional[Language]:
        """Detect programming language from file extension"""
        ext_map = {
            '.py': Language.PYTHON,
            '.js': Language.JAVASCRIPT,
            '.java': Language.JAVA,
            '.cs': Language.CSHARP,
            '.go': Language.GO
        }
        return ext_map.get(Path(file_path).suffix.lower())

    @staticmethod
    def build_dependency_graph(project_root: str) -> Dict:
        """Multi-language dependency graph builder"""
        graph = {}
        for root, _, files in os.walk(project_root):
            for file in files:
                path = os.path.join(root, file)
                lang = CodeAnalysisTools.detect_language(path)
                if not lang:
                    continue

                with open(path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    imports = []

                    if lang == Language.PYTHON:
                        imports = CodeAnalysisTools._extract_python_imports(content)
                    elif lang == Language.JAVASCRIPT:
                        imports = CodeAnalysisTools._extract_js_imports(content)
                    elif lang == Language.JAVA:
                        imports = CodeAnalysisTools._extract_java_imports(content)

                    graph[path] = {
                        'language': lang.name,
                        'imports': imports
                    }
        return graph

    @staticmethod
    def _extract_python_imports(code: str) -> List[str]:
        """Python-specific import extraction"""
        imports = []
        try:
            tree = ast.parse(code)
            for node in ast.walk(tree):
                if isinstance(node, ast.Import):
                    imports.extend(alias.name for alias in node.names)
                elif isinstance(node, ast.ImportFrom):
                    imports.extend(f"{node.module}.{alias.name}" for alias in node.names)
        except:
            pass
        return imports

    @staticmethod
    def _extract_js_imports(code: str) -> List[str]:
        """JavaScript-specific import extraction"""
        imports = []
        # Match ES6 imports
        imports.extend(re.findall(r'import\s+(?:\{[^}]+\}|\w+)\s+from\s+[\'"]([^\'"]+)[\'"]', code))
        # Match require() calls
        imports.extend(re.findall(r'require\([\'"]([^\'"]+)[\'"]\)', code))
        return imports

    @staticmethod
    def _extract_java_imports(code: str) -> List[str]:
        """Java-specific import extraction"""
        return re.findall(r'import\s+([\w\.]+)\s*;', code)

class CodeGenius:
    def __init__(self, project_root: str):
        self.project_root = project_root
        self.context = self._load_project_context()
        self.user_prefs = {
            "coding_style": "standard",
            "language": "auto",
            "domain": "general"
        }
        self.dependency_graph = CodeAnalysisTools.build_dependency_graph(project_root)

    def _load_project_context(self) -> Dict:
        """Multi-language project context loader"""
        context = {
            "files": {},
            "business_rules": [],
            "languages_used": set()
        }

        for root, _, files in os.walk(self.project_root):
            for file in files:
                path = Path(root) / file
                lang = CodeAnalysisTools.detect_language(path)
                if not lang:
                    continue

                with open(path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    context["files"][str(path)] = {
                        "content": content,
                        "language": lang.name
                    }
                    context["languages_used"].add(lang.name)
        return context

    def generate_code(self, prompt: str, file_path: str) -> Dict:
        """Generate code using Gemini 2.0 Flash"""
        lang = CodeAnalysisTools.detect_language(file_path)
        if not lang:
            return {"error": "Unsupported file type"}

        full_prompt = f"""
        [Project Context]
        Languages Used: {', '.join(self.context['languages_used'])}
        Target Language: {lang.name}

        [Task]
        {prompt}

        [Requirements]
        1. Generate {lang.name} code matching project standards
        2. Include language-appropriate error handling
        3. Add security safeguards for {lang.name}
        4. Follow best practices for {lang.name}
        5. Provide {self._get_doc_style(lang)}-style documentation
        """

        try:
            response = model.generate_content(full_prompt)
            generated_text = response.text
        except Exception as e:
            generated_text = f"// Error generating code: {str(e)}"

        return self._process_response(generated_text, file_path, lang)

    def _process_response(self, raw_response: str, file_path: str, lang: Language) -> Dict:
        """Process the raw response from the AI model"""
        code = self._extract_code_blocks(raw_response, lang)

        return {
            "code": code,
            "language": lang.name,
            "documentation": self._generate_ai_docs(code, lang),
            "tests": self._generate_tests(code, lang),
        }

    def _extract_code_blocks(self, text: str, lang: Language) -> str:
        """Extract code blocks from the raw AI response"""
        lang_tags = {
            Language.PYTHON: 'python',
            Language.JAVASCRIPT: 'javascript',
            Language.JAVA: 'java',
            Language.CSHARP: 'csharp',
            Language.GO: 'go'
        }
        tag = lang_tags.get(lang, 'python')
        pattern = fr'```(?:{tag})?\n(.*?)\n```'
        code_blocks = re.findall(pattern, text, re.DOTALL)
        return '\n'.join(code_blocks) if code_blocks else text

    def _generate_ai_docs(self, code: str, lang: Language) -> str:
        """Generate AI-powered documentation for the generated code"""
        doc_style = self._get_doc_style(lang)
        prompt = f"""
        Generate {doc_style}-style documentation for this {lang.name} code:
        {code}
        Include:
        1. Module/class-level documentation
        2. Function/method docstrings
        3. Important inline comments
        """
        return model.generate_content(prompt).text

    def _get_doc_style(self, lang: Language) -> str:
        """Get documentation style for language"""
        styles = {
            Language.PYTHON: "Google",
            Language.JAVASCRIPT: "JSDoc",
            Language.JAVA: "JavaDoc",
            Language.CSHARP: "XML",
            Language.GO: "GoDoc"
        }
        return styles.get(lang, "standard")

    def _generate_tests(self, code: str, lang: Language) -> str:
        """Generate tests based on the generated code"""
        prompt = f"""
        Generate comprehensive tests for this {lang.name} code:
        {code}
        Include:
        1. Unit tests with edge cases
        2. Integration tests if applicable
        3. Performance tests if needed
        Use appropriate testing framework for {lang.name}
        """
        return model.generate_content(prompt).text

def generate_code_for_ui(project_dir, prompt, filename, user_language):
    """Generate code and output details from the CodeGenius assistant."""
    # Initialize the assistant
    assistant = CodeGenius(project_dir)

    # Set language preference from user input
    if user_language != "auto":
        assistant.user_prefs["language"] = user_language.lower()

    # Generate code
    file_path = os.path.join(project_dir, filename)
    result = assistant.generate_code(prompt, file_path)

    # Format the output
    output = f"""=== GENERATED CODE ===
Language: {result['language']}

--- Code ---
{result['code']}

--- Documentation ---
{result['documentation']}

--- Tests ---
{result['tests']}
"""
    return output

def save_uploaded_files(files):
    """Save uploaded files to a temporary directory and return the directory path"""
    temp_dir = "temp_project"
    os.makedirs(temp_dir, exist_ok=True)

    for file in files:
        with open(os.path.join(temp_dir, file.name), "wb") as f:
            f.write(file.read())

    return temp_dir

def get_file_choices(files):
    """Return list of files uploaded by the user."""
    if not files:
        return []
    return [file.name for file in files]

def create_gradio_interface():
    with gr.Blocks(title="CodeGenius AI Assistant") as demo:
        gr.Markdown("""
        # CodeGenius AI Assistant
        Generate production-ready code with documentation and tests for multiple languages
        """)

        with gr.Row():
            with gr.Column():
                file_upload = gr.File(
                    label="Upload Project Files",
                    file_count="multiple",
                    file_types=[".py", ".js", ".java", ".cs", ".go"],
                    type="filepath"
                )
                language_select = gr.Dropdown(
                    label="Target Language",
                    choices=["auto", "Python", "JavaScript", "Java", "C#", "Go"],
                    value="auto"
                )
                file_select = gr.Dropdown(
                    label="Select File (optional)",
                    choices=[],
                    interactive=True
                )
                prompt_input = gr.Textbox(
                    label="Code Generation Prompt",
                    placeholder="Describe the code you want to generate...",
                    lines=3
                )
                generate_btn = gr.Button("Generate Code", variant="primary")

            with gr.Column():
                output_code = gr.Code(
                    label="Generated Code",
                    language="python",
                    interactive=True
                )
                output_docs = gr.Code(
                    label="Documentation",
                    language="markdown",
                    interactive=True
                )
                output_tests = gr.Code(
                    label="Generated Tests",
                    interactive=True
                )

        # Event handlers
        file_upload.upload(
            fn=get_file_choices,
            inputs=file_upload,
            outputs=file_select
        )

        generate_btn.click(
            fn=lambda files, prompt, filename, lang: generate_code_for_ui(
                save_uploaded_files(files),
                prompt,
                filename,
                lang
            ),
            inputs=[file_upload, prompt_input, file_select, language_select],
            outputs=[output_code, output_docs, output_tests]
        )

    return demo

if __name__ == "__main__":
    demo = create_gradio_interface()
    demo.launch(share=True)